In [ ]:
import pandas as pd

df = pd.read_csv("final_learning_potential_dataset.csv")


Veri seti boyutu: (427, 7)
Sütunlar: ['stress_congeneric_score', 'control_congeneric', 'self_efficacy_congeneric', 'depression_congeneric_score', 'somatic_congeneric', 'cog_affect_congeneric', 'learning_potential_score']

Son 3 satır:


,stress_congeneric_score,control_congeneric,self_efficacy_congeneric,depression_congeneric_score,somatic_congeneric,cog_affect_congeneric,learning_potential_score
424,-0.665961,1.056399,-0.159883,-0.092050,-0.186550,0.052179,0.693036
425,1.113904,-0.781303,-1.639937,0.892721,1.781275,0.094988,-2.336883
426,0.192037,-0.037275,-0.342274,-0.318983,-0.335821,-0.294627,0.165029


In [3]:
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression, Ridge, Lasso 
from sklearn.ensemble import RandomForestRegressor
import numpy as np 


In [4]:
feature_cols=  [
    "stress_congeneric_score","control_congeneric","self_efficacy_congeneric",
    "depression_congeneric_score","somatic_congeneric","cog_affect_congeneric"
]

X = df[feature_cols].copy()
y = df["learning_potential_score"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

In [7]:
models = {
    "Linear": Pipeline([("scaler", StandardScaler()), ("model", LinearRegression())]),
    "Ridge": Pipeline([("scaler", StandardScaler()), ("model", Ridge(alpha=1.0, random_state=42))]),
    "Lasso": Pipeline([("scaler", StandardScaler()), ("model", Lasso(alpha=1.0, random_state=42))])
}

In [9]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
rows = []
for name, mdl in models.items():
    scores = cross_validate(mdl, X_train, y_train, 
                            cv = cv, scoring= ("r2", "neg_mean_absolute_error", "neg_root_mean_squared_error"), 
                            return_train_score=False)
    rows.append({
        "model": name,
        "cv_R2_mean": np.mean(scores["test_r2"]),
        "cv_MAE_mean": -np.mean(scores["test_neg_mean_absolute_error"]),
        "cv_RMSE_mean": -np.mean(scores["test_neg_root_mean_squared_error"])
    })

In [15]:
results = pd.DataFrame(rows).sort_values("cv_R2_mean" ,ascending=False)
results

,model,cv_R2_mean,cv_MAE_mean,cv_RMSE_mean
0,Linear,1.000000,9.913940e-16,1.208637e-15
1,Ridge,0.999999,1.391539e-03,1.690324e-03
2,Lasso,0.731855,8.950326e-01,1.089155e+00


r2 skoru bize gösteriyor ki pca doğru çalışmış ve learning potential score başarılı oluşturulmuş